# Введение в обработку естественного языка
## Курсовой проект

In [ ]:
!pip install python-telegram-bot --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513 kB 33.6 MB/s 
     |████████████████████████████████| 423 kB 72.3 MB/s 
     |████████████████████████████████| 58 kB 7.3 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.4
    Uninstalling cachetools-4.2.4:
      Successfully uninstalled cachetools-4.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0, but you have tornado 6.2 which is incompatible.


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.6 MB/s 
     |████████████████████████████████| 8.2 MB 56.2 MB/s 


In [ ]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=60e28fb7f4f825f38563da02a83b9ddfdb4486687015daeb4b5a2ac674025273
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [ ]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 646 kB 32.7 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391563 sha256=8c3dfd98c2a9bb032f5c6629ba65ad0a5edf154c7846084af2358a05cbbf3c0d
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [ ]:
import numpy as np
import pandas as pd

import os
import annoy
import pickle
import string

from tqdm.notebook import tqdm_notebook
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from telegram.ext  import Updater, CommandHandler, MessageHandler, filters
from telegram import Animation

ImportError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
products_dataset = pd.read_csv("/content/drive/My Drive/ProductsDataset.csv")
input_file = "/content/drive/My Drive/Otvety.txt"
output_file = "/content/drive/My Drive/prepared_answers.txt"

In [ ]:
def preprocess_txt(line):
    """Строковый препроцессинг текстовой строки"""
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [ ]:
assert True

#Small preprocess of the answers

question = None
written = False

with open(output_file, "w", encoding="utf8") as fout:
    with open(input_file, "r", encoding="utf8") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [ ]:
%%time

assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open(input_file, "r", encoding="utf8") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

CPU times: user 35min 40s, sys: 11.6 s, total: 35min 52s
Wall time: 35min 54s


In [ ]:
sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
modelFT.save("ft_model")

In [ ]:
%%time

modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open(output_file, "r", encoding="utf8") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save('speaker.ann')

0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


CPU times: user 1h 27min 51s, sys: 51.1 s, total: 1h 28min 43s
Wall time: 1h 28min 8s


In [ ]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')

True

In [ ]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[46005, 86221]

In [ ]:
products_dataset['text'] = products_dataset['title'] + " " + products_dataset["descrirption"]
products_dataset['text'] = products_dataset['text'].apply(lambda x: preprocess_txt(str(x)))
products_dataset.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

In [ ]:
idxs = set(np.random.randint(0, len(index_map), len(products_dataset)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in products_dataset['text'].values]

In [ ]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, 
                                                    stratify=labels, random_state=13)

In [ ]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [ ]:
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9813660195550518

In [ ]:
assert True

with open("/content/drive/My Drive/Colab Notebooks/index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [ ]:
tfidf_vect = TfidfVectorizer().fit(X_train)

In [ ]:
np.mean(tfidf_vect.idf_)

10.92054728927344

In [ ]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'девочка': 9.543339292634592,
 'нравиться': 6.957627943724626,
 'самый': 9.995324416377649,
 'написать': 11.248087384873017,
 'десятка': 10.842622276764853,
 'письмо': 11.248087384873017,
 'служба': 11.248087384873017,
 'поддержка': 11.248087384873017,
 'удаться': 11.248087384873017,
 'выпытать': 10.842622276764853,
 'ответ': 11.248087384873017,
 'поменять': 10.331796652998863,
 'пароль': 11.248087384873017,
 'выслать': 11.248087384873017,
 'почтовый': 11.248087384873017,
 'ящик': 11.248087384873017,
 'порекомендовать': 11.248087384873017,
 'сменить': 10.842622276764853,
 'проц': 11.248087384873017,
 'двухъядерный': 11.248087384873017,
 'торрент': 11.248087384873017,
 'искать': 11.248087384873017,
 'сыворотка': 11.248087384873017,
 'лицо': 10.842622276764853,
 'гиалуроновый': 11.248087384873017,
 'кислота': 10.842622276764853,
 'обеспечивать': 11.248087384873017,
 'увлажнение': 10.554940204313072,
 'кожа': 11.248087384873017,
 'течение': 11.248087384873017,
 'час': 11.248087384873017,

In [ ]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(products_dataset))):
    n_ft = 0
    index_map_shop[counter] = (products_dataset.loc[i, "title"], products_dataset.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in products_dataset.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

  0%|          | 0/35548 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


True

In [ ]:
assert True

with open("/content/drive/My Drive/Colab Notebooks/index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [ ]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([12], [1.4142135381698608])

In [ ]:
!pip install telegram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached telegram-0.0.1-py3-none-any.whl


In [ ]:
!pip uninstall tornado
!pip install tornado==4.5.3

Found existing installation: tornado 6.2
Uninstalling tornado-6.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/tornado-6.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tornado/*
Proceed (y/n)? y
  Successfully uninstalled tornado-6.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 484 kB 29.9 MB/s 
  Created wheel for tornado: filename=tornado-4.5.3-cp37-cp37m-linux_x86_64.whl size=434039 sha256=7653feb9653574670c40e0f381f710abaa6bbd73d627499fc8c4eb79be8b3c88
  Stored in directory: /root/.cache/pip/wheels/a2/45/43/36ec7a893e16c1212a6b1505ded0a2d73cf8e863a0227c8e04
Successfully built tornado
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-telegram-bot 13.13 requires tornado>=6.1, but you have tornado 4.5.3 which is incompatible.
google-colab

In [ ]:
!pip uninstall telegram==0.0.1 

Found existing installation: telegram 0.0.1
Uninstalling telegram-0.0.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/telegram-0.0.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/telegram/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/telegram/__main__.py
    /usr/local/lib/python3.7/dist-packages/telegram/base.py
    /usr/local/lib/python3.7/dist-packages/telegram/bot.py
    /usr/local/lib/python3.7/dist-packages/telegram/botcommand.py
    /usr/local/lib/python3.7/dist-packages/telegram/botcommandscope.py
    /usr/local/lib/python3.7/dist-packages/telegram/callbackquery.py
    /usr/local/lib/python3.7/dist-packages/telegram/chat.py
    /usr/local/lib/python3.7/dist-packages/telegram/chataction.py
    /usr/local/lib/python3.7/dist-packages/telegram/chatadministratorrights.py
    /usr/local/lib/python3.7/dist-packages/telegram/chatinvitelink.py
    /usr/local/lib/python3.7/dist-packages/telegram/chatjoinrequest.py
    /u

In [ ]:
start_msg = 'Привет. Я бот. Бот Василий Гикбрейнович. Вселяю в вас уверенность, что вы прошли обучение почти полностью. Чем могу помочь?'

In [ ]:
updater = Updater(token='5533294116:AAGAh_cKbwFSqvrNaJ0pO4BFlQRkFd0GfLk')
dispatcher = updater.dispatcher

NameError: ignored

In [ ]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text=start_msg)

In [ ]:
def textMessage(bot, update):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        print(distances[0])
        bot.send_message(chat_id=update.message.chat_id, text="Чаво ?")
        return
    bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])


In [ ]:
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()